<a href="https://colab.research.google.com/github/Mohamed-ux-beep/Image-Classification/blob/main/image_classi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import shutil
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import *
from ray import tune
from ray.air import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler

2023-12-01 16:11:26,906	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-01 16:11:27,766	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
tra_path = "/kaggle/input/train-2-zip/train_2"
val_path = "/kaggle/input/val-2-zip/val_2"

# **Dataset for Training & Validation data**

In [ ]:
def load_data(tra_path, val_path):
    transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = datasets.ImageFolder(root=tra_path, transform=transform)
    validset = datasets.ImageFolder(root=val_path, transform=transform)
    return trainset, validset

# **The neural network**

In [ ]:
class CNN(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3,   64,  3)
        self.conv2 = nn.Conv2d(64,  128, 3)
        self.conv3 = nn.Conv2d(128, 256, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 4 * 4, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)

In [ ]:
model = CNN()

In [ ]:
total = 0
print('Trainable parameters:')
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, '\t', param.numel())
        total += param.numel()
print()
print('Total', '\t', total)

Trainable parameters:
conv1.weight 	 1728
conv1.bias 	 64
conv2.weight 	 73728
conv2.bias 	 128
conv3.weight 	 294912
conv3.bias 	 256
fc1.weight 	 131072
fc1.bias 	 128
fc2.weight 	 32768
fc2.bias 	 256
fc3.weight 	 2304
fc3.bias 	 9

Total 	 537353


# **Training of the model**

In [ ]:
def train(model, train_loader, optimizer, epoch):
    losses = []
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        if batch_idx > 0 and batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{}\t({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return losses

# **Validation of the model**

In [ ]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return (float(correct) / len(test_loader.dataset))

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cpu") # or 'gpu'
losses = []
accuracies = []
for epoch in range(0, 20):
    losses.extend(train(model, tr_dataloader, optimizer, epoch))
    accuracies.append(test(model,tr_dataloader))

Train Epoch: 0 [4000/45000	(9%)]	Loss: 2.158592
Train Epoch: 0 [8000/45000	(18%)]	Loss: 1.358586
Train Epoch: 0 [12000/45000	(27%)]	Loss: 1.873774
Train Epoch: 0 [16000/45000	(36%)]	Loss: 1.179891
Train Epoch: 0 [20000/45000	(44%)]	Loss: 1.041986
Train Epoch: 0 [24000/45000	(53%)]	Loss: 1.576380
Train Epoch: 0 [28000/45000	(62%)]	Loss: 2.063609
Train Epoch: 0 [32000/45000	(71%)]	Loss: 2.666649
Train Epoch: 0 [36000/45000	(80%)]	Loss: 1.767159
Train Epoch: 0 [40000/45000	(89%)]	Loss: 1.378502
Train Epoch: 0 [44000/45000	(98%)]	Loss: 1.603498

Average loss: 1.2643, Accuracy: 23855/45000 (53%)

Train Epoch: 1 [4000/45000	(9%)]	Loss: 0.788922
Train Epoch: 1 [8000/45000	(18%)]	Loss: 1.222013
Train Epoch: 1 [12000/45000	(27%)]	Loss: 0.504328
Train Epoch: 1 [16000/45000	(36%)]	Loss: 0.996005
Train Epoch: 1 [20000/45000	(44%)]	Loss: 1.158540
Train Epoch: 1 [24000/45000	(53%)]	Loss: 1.087369
Train Epoch: 1 [28000/45000	(62%)]	Loss: 0.647539
Train Epoch: 1 [32000/45000	(71%)]	Loss: 0.360332
Trai

In [ ]:
def test_label_predictions(model,test_loader):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            prediction = output.argmax(dim=1, keepdim=True)
            actuals.extend(target.view_as(prediction))
            predictions.extend(prediction)
    return [i.item() for i in actuals], [i.item() for i in predictions]

actuals, predictions = test_label_predictions(model,val_dataloader)
print('Confusion matrix:')
print(confusion_matrix(actuals, predictions))
print('F1 score: %f' % f1_score(actuals, predictions, average='micro'))
print('Accuracy score: %f' % accuracy_score(actuals, predictions))

Confusion matrix:
[[774  23  86  13  13   9  19  37  26]
 [ 16 862  10  12   1  10   3  23  63]
 [ 44   8 696  65  71  63  41   8   4]
 [ 30  13 104 540  51 184  40  11  27]
 [ 27   4 112  70 640  55  78  10   4]
 [ 15   6  74 161  29 658  45   6   6]
 [ 23   4  37  46  53  65 757   1  14]
 [ 77  27  25  14   6  11   7 798  35]
 [ 36 107   6  18   9  10  12  23 779]]
F1 score: 0.722667
Accuracy score: 0.722667


In [ ]:
tst_path = "/kaggle/input/test-data/test"

In [ ]:
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
testset = datasets.ImageFolder(root=tst_path, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

In [ ]:
predictions = []
with torch.no_grad():
    for images, _ in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.tolist())


In [ ]:
import os
# Initialize an empty list to store the file paths
image_paths = []
# Iterate over the files in the directory and append their paths to the list
for root, dirs, files in os.walk(tst_path):
    for file in files:
        if file.endswith(".jpg"):
            file_path = os.path.join(root, file)
            image_paths.append(file_path)



In [ ]:
import pandas as pd

In [ ]:
results = pd.DataFrame(columns=['image', 'class', 'id'])

In [ ]:
results['image'] = image_paths

In [ ]:
results['id'] = predictions

In [ ]:
class_dict = {0:'airplan', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'horse', 7:'ship', 8:'truck'}

In [ ]:
results['class'] = results['id'].apply(lambda x: class_dict[x])

In [ ]:
results.to_csv('img_classification_1.csv', index=0)

In [ ]:
pd.read_csv('img_classification_1.csv')

,image,class,id
0,/kaggle/input/test-data/test/0664.jpg,airplan,0
1,/kaggle/input/test-data/test/1269.jpg,ship,7
2,/kaggle/input/test-data/test/3863.jpg,truck,8
3,/kaggle/input/test-data/test/2193.jpg,horse,6
4,/kaggle/input/test-data/test/0733.jpg,horse,6
...,...,...,...
4495,/kaggle/input/test-data/test/4463.jpg,ship,7
4496,/kaggle/input/test-data/test/3950.jpg,deer,4
4497,/kaggle/input/test-data/test/1656.jpg,automobile,1
4498,/kaggle/input/test-data/test/3361.jpg,deer,4
